# <center>Workflow for on-line GC and HPLC analysis in flow chemistry</center>


---
## Section 0: Imports, Paths, and Logging
---

In this section all the necessary python packages are imported, the path to this notebook and the logger for this notebook is set up.

In [1]:
# Activate autoreload to keep on track with changing modules #
%reload_ext autoreload
%autoreload 2

# Import standard libraries #
import numpy as np
import logging
import json
from pathlib import Path

# Import tools for data processing and analysis and DaRUS upload #
from datamodel.tools import initialize_dataset, reading_raw_data_widget, analyzing_raw_data_widget, DaRUS_upload

In [2]:
# Define paths for loggin output #
root                = Path.cwd()
logging_config_path = root / "datamodel/tools/logger_config.json"

# Read in logger specs and configurate logger (set name to current notebook) #
with open(logging_config_path) as logging_config_json: logging.config.dictConfig( json.load( logging_config_json ) )
logger = logging.getLogger(__name__)

# Set the level of thid-party logger to avoid dumping too much information #
for logger_ in ['markdown_it', 'h5py', 'numexpr', 'git']: logging.getLogger(logger_).setLevel('WARNING')

---
## Section 1: Dataset and raw data
---
In this section the dataset as well as the to analyze raw data is choosen

In [3]:
git_path = 'https://github.com/FAIRChemistry/datamodel_b07_tc.git'
branch   = 'samir_develop'

id = initialize_dataset()
id.write_dataset(root, git_path, branch)

Saving dataset!


In [4]:
## Definition of basic meta data of the project ##

id.title.value        = 'Electrocatalytic CO2-reduction on carbon'
id.description.value  = 'The aim of this project is to blablabla'
id.project.value      = 'Project B07'

# List with authors and their affiliation #
id.authors.value      = 'Richard Schömig, Maximilian Schmidt' 
id.affiliations.value = 'University of Stuttgart, University of Stuttgart'
id.identifier.value   = 'xxx-xxx-xxx-xxx, xxx-xxx-xxx-xxx'
id.contact_text.value = 'Richard Schömig, richard@web.de'

id.related_publication.value = "test, https://id.loc.gov/authorities/subjects/sh2014001146.html"

id.topic_classification.value = "homogeneous catalysis (LCSH), https://id.loc.gov/authorities/subjects/sh2014001146.html"
id.keywords.value             = "polymer chemistry (Loterre Chemistry Vocabulary), https://skosmos.loterre.fr/ERC/en/page/?uri=http%3A%2F%2Fdata.loterre.fr%2Fark%3A%2F67375%2FERC-KCSKD4X9-P"

id.dataset_text.value = "test"

In [3]:
## Search for dataset and raw data ##

rrdw = reading_raw_data_widget()
rrdw.choose_data(root)

In [6]:
#str(root) = "c:/Users/darouich/OneDrive/Dokumente/"

e_chem = str(root)+'/data/Rohdaten/01_EChem/CAD14-Cu@AB/GSTATIC.DTA'
mfm    = str(root)+'/data/Rohdaten/03_MFM/CAD14-Cu@AB/Bench-2h-GSS_CAD14-Cu@AB_200_50c_24h_truncated.csv'
gc     = [str(root)+'/data/Rohdaten/02_GC/CAD14-Cu@AB/JH-1H 2023-02-06 10-00-18/NV-F0102.D/report00.CSV',
        str(root)+'/data/Rohdaten/02_GC/CAD14-Cu@AB/JH-1H 2023-02-06 10-00-18/NV-F0102.D/REPORT01.CSV',
        str(root)+'/data/Rohdaten/02_GC/CAD14-Cu@AB/JH-1H 2023-02-06 10-00-18/NV-F0103.D/report00.CSV',
        str(root)+'/data/Rohdaten/02_GC/CAD14-Cu@AB/JH-1H 2023-02-06 10-00-18/NV-F0103.D/REPORT01.CSV',
        str(root)+'/data/Rohdaten/02_GC/CAD14-Cu@AB/JH-1H 2023-02-06 10-00-18/NV-F0104.D/report00.CSV',
        str(root)+'/data/Rohdaten/02_GC/CAD14-Cu@AB/JH-1H 2023-02-06 10-00-18/NV-F0104.D/REPORT01.CSV']
calib  = str(root)+'/data/calibration/calibration.json'
correc = str(root)+'/data/correction_factors/correction_factors.json'
farada = str(root)+'/data/faraday_coefficients/faraday_coefficients.json'

rrdw.Echem_files.value = [e_chem]
rrdw.MFM_files.value   = [mfm]
rrdw.GC_files.value    = gc
rrdw.calib_files.value = [calib]
rrdw.correction_files.value = [correc]
rrdw.faraday_files.value    = [farada]

---
## Section 2: Analysis of data
---
In this section the raw data of the above choosen dataset is analyzed (if you change the dataset above, then reexecute this cell)

In [6]:
# Provide a typical retention time dictionary to pre assign retention times 

typical_retention_time = {"Hydrogen": 1.7, "Carbon dioxide": 3.0, "Carbon monoxide": 13.6, 
                          "Methane": 3.6, "Ethene": 6.0, "Ethane": 7.1}

ardw = analyzing_raw_data_widget()
ardw.choose_experiment( datamodel = rrdw.datamodel, 
                        dataset_path = rrdw.dataset_dropdown.value, 
                        typical_retention_time = typical_retention_time)

---
## Section 3: Upload of data to DaRUS
---
In this section the dataset containing the processed as well as the raw data, is uploaded to DaRUS

In [49]:
sfb1333_dataverse_list = [ "sfb1333-hansen-gross" ]

api_token = "4afecd82-c92d-4935-b786-2225af43531e"

test = DaRUS_upload()

test.upload( datamodel = rrdw.datamodel, 
             dataset_path = rrdw.dataset_dropdown.value,
             dataverse_list = sfb1333_dataverse_list )

#test.depositor_text.value = "samir Darouich"
test.api_token_text.value = api_token

Attribute project not valid for import (dv_up).


Exception: Validation Failed: Depositor is required. (Invalid value:edu.harvard.iq.dataverse.DatasetField[ id=null ]).java.util.stream.ReferencePipeline$3@6126ae76

TypeError: DaRUS_upload.update_to_DaRUS() takes 1 positional argument but 2 were given

In [13]:
Dataset.from_dataverse_doi('doi:10.18419/darus-3812', filedir=".", dataverse_url='https://darus.uni-stuttgart.de', api_token=api_token)

ApiUrlError: base_url None is not a string.